In [3]:
! pip install torch
! pip install torchtext

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

OSError: [WinError 127] 지정된 프로시저를 찾을 수 없습니다

In [5]:
#로컬 모듈 import
import sys
import os
print(os.getcwd())
# 모듈 경로를 sys.path에 추가
module_path = os.path.abspath(os.path.join('..', 'my_transformer'))
if module_path not in sys.path:
    sys.path.append(module_path)
from my_transformer.my_transformer import Transformer

c:\Users\박동연\Documents\GitHub\24-2_DS_assignment\Week_01\Transformer


In [163]:
# 1. IMDb 데이터셋 로드 및 전처리
tokenizer = get_tokenizer("basic_english")

# IMDb 데이터셋에서 리뷰와 라벨을 분리
def yield_tokens(data_iter):
    for label, line in data_iter:
        yield tokenizer(line)

# IMDb 데이터셋 로드
train_iter, test_iter = IMDB(split=('train', 'test'))

# 토큰화를 거친 후 단어 사전(vocab) 생성
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# 텍스트를 텐서로 변환
def text_pipeline(x):
    return vocab(tokenizer(x))

# 라벨을 텐서로 변환 (0 또는 1)
def label_pipeline(x):
    return 1 if x == "pos" else 0

# IMDb 데이터를 텐서로 변환 및 패딩
def collate_batch(batch):
    label_list, text_list = [], []
    for _label, _text in batch:
        label_list.append(torch.tensor(label_pipeline(_label), dtype=torch.long))
        text_list.append(torch.tensor(text_pipeline(_text), dtype=torch.long))
    return torch.stack(label_list), pad_sequence(text_list, batch_first=True)

# DataLoader 생성
batch_size = 16
train_iter, test_iter = IMDB(split=('train', 'test'))
train_dataloader = DataLoader(list(train_iter), batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(list(test_iter), batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

In [164]:
# 2. 트랜스포머 모델 정의
# UCI Wine 데이터셋의 특성에 맞게 트랜스포머 모델 초기화
src_vocab_size = 13  # 입력 특성 개수
tgt_vocab_size = 3  # 분류 클래스 개수
d_model = 16  # 임베딩 차원
n_heads = 2  # 멀티헤드 어텐션의 헤드 수
d_ff = 32  # 피드포워드 레이어의 차원
num_encoder_layers = 2  # 인코더 레이어 수
num_decoder_layers = 2  # 디코더 레이어 수
dropout = 0.1  # 드롭아웃 비율

model = Transformer(src_vocab_size, tgt_vocab_size, d_model, n_heads, d_ff, num_encoder_layers, num_decoder_layers, dropout)

# 3. 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류이므로 CrossEntropyLoss 사용
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. 모델 학습 함수
def train_model(model, criterion, optimizer, train_loader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # 입력 데이터를 모델에 맞게 형식 변환
            inputs = inputs.unsqueeze(1)  # Transformer의 입력 형태 [batch_size, seq_len, d_model]
            labels = labels.unsqueeze(1)

            # 옵티마이저 초기화
            optimizer.zero_grad()

            # 모델의 예측 결과
            outputs = model(inputs, labels)

            # 손실 계산
            loss = criterion(outputs.squeeze(), labels.squeeze())

            # 역전파 및 가중치 업데이트
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# 5. 모델 테스트 함수
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.unsqueeze(1)
            labels = labels.unsqueeze(1)

            outputs = model(inputs, labels)
            _, predicted = torch.max(outputs, 1)  # 가장 높은 값의 인덱스를 예측 값으로 사용
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

In [165]:
# 6. 모델 학습 및 테스트 실행
num_epochs = 20
train_model(model, criterion, optimizer, train_loader, num_epochs)
test_model(model, test_loader)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [158]:
class PatternDataset(Dataset):
    def __init__(self, num_samples=1000, sequence_length=4, max_num=5):
        self.data = []
        self.targets = []
        for _ in range(num_samples):
            start = torch.randint(0, max_num, (1,)).item()  # 시작 숫자
            diff = torch.randint(1, 5, (1,)).item()  # 등차
            sequence = [start + i * diff for i in range(sequence_length)]
            next_value = sequence[-1] + diff  # 다음에 올 숫자
            
            # 입력 시퀀스는 정수로 이루어진 시퀀스, 타겟은 다음에 올 숫자
            self.data.append(torch.tensor(sequence))
            self.targets.append(next_value)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# 하이퍼파라미터 설정
src_vocab_size = 17
tgt_vocab_size = 21
d_model = 32
n_heads = 12
d_ff = 64
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.1
batch_size = 32
num_epochs = 50
learning_rate = 0.0005

# 데이터셋 및 데이터로더 생성
dataset = PatternDataset(num_samples=1000)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 모델 초기화
model = Transformer(src_vocab_size, tgt_vocab_size, d_model, n_heads, d_ff, num_encoder_layers, num_decoder_layers, dropout)

# 손실 함수 및 최적화 도구 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for src, tgt in dataloader:
        # 입력 시퀀스 준비
        tgt_input = torch.zeros_like(tgt).unsqueeze(1)  # 예측 시작을 위한 빈 타겟 시퀀스
        tgt = tgt.unsqueeze(1)  # 타겟을 2D 텐서로 변환
        
        optimizer.zero_grad()

        # 모델에 입력 및 타겟 시퀀스 전달
        output = model(src, tgt_input)

        # 출력 크기 조정 및 손실 계산
        output = output.view(-1, tgt_vocab_size)  # (batch_size, vocab_size)
        tgt = tgt.view(-1)  # (batch_size)
        
        loss = criterion(output, tgt)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")

Epoch 1/50, Loss: 2.9195
Epoch 2/50, Loss: 2.8510
Epoch 3/50, Loss: 2.8301
Epoch 4/50, Loss: 2.8120
Epoch 5/50, Loss: 2.8092
Epoch 6/50, Loss: 2.8139
Epoch 7/50, Loss: 2.8042
Epoch 8/50, Loss: 2.8055
Epoch 9/50, Loss: 2.7988
Epoch 10/50, Loss: 2.7962
Epoch 11/50, Loss: 2.7844
Epoch 12/50, Loss: 2.7993
Epoch 13/50, Loss: 2.7908
Epoch 14/50, Loss: 2.7954
Epoch 15/50, Loss: 2.7896
Epoch 16/50, Loss: 2.7952
Epoch 17/50, Loss: 2.7868
Epoch 18/50, Loss: 2.7917
Epoch 19/50, Loss: 2.7909
Epoch 20/50, Loss: 2.7876
Epoch 21/50, Loss: 2.7928
Epoch 22/50, Loss: 2.7876
Epoch 23/50, Loss: 2.7896
Epoch 24/50, Loss: 2.7924
Epoch 25/50, Loss: 2.7921
Epoch 26/50, Loss: 2.7921
Epoch 27/50, Loss: 2.7909
Epoch 28/50, Loss: 2.7885
Epoch 29/50, Loss: 2.7858
Epoch 30/50, Loss: 2.7806
Epoch 31/50, Loss: 2.7896
Epoch 32/50, Loss: 2.7907
Epoch 33/50, Loss: 2.7766
Epoch 34/50, Loss: 2.7888
Epoch 35/50, Loss: 2.7920
Epoch 36/50, Loss: 2.7914
Epoch 37/50, Loss: 2.7936
Epoch 38/50, Loss: 2.7852
Epoch 39/50, Loss: 2.

In [ ]:
# 테스트 데이터셋 생성 (학습 데이터와 동일한 방식으로 생성)
test_dataset = PatternDataset(num_samples=1000)  # 테스트용 샘플 수
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

def test_model(model, dataloader):
    model.eval()  # 평가 모드로 설정
    total_correct = 0
    total_samples = 0

    with torch.no_grad():  # 그래디언트 계산 중지 (평가 시에는 필요하지 않음)
        for src, tgt in dataloader:
            # 입력 시퀀스 준비
            tgt_input = torch.zeros_like(tgt).unsqueeze(1)  # 예측 시작을 위한 빈 타겟 시퀀스
            tgt = tgt.unsqueeze(1)  # 타겟을 2D 텐서로 변환
            
            # 모델에 입력 시퀀스를 전달하고 예측 값 생성
            output = model(src, tgt_input)
            predicted = output.argmax(dim=-1)  # 예측 결과는 argmax를 통해 얻음
            
            # 실제 타겟과 예측값 비교
            correct = (predicted.view(-1) == tgt.view(-1)).sum().item()
            total_correct += correct
            total_samples += tgt.size(0)

    # 정확도 계산
    accuracy = total_correct / total_samples * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# 학습된 모델 테스트
test_model(model, test_dataloader)


Test Accuracy: 9.20%
